# **Probing Tasks - How Does BERT Answer Questions?**
In this notebook, we will carry out the following badges:

1.   reproduce the probing tasks:
  *   NEL, REL, COREF on the Ontonotes dataset
  *   QUES on TREC-10 dataset
  *   SUP on the Squad dataset


2.   experiment with BERT (**todo:** which type?) trained on (**todo:** which dataset?) dataset
3.   experiment with T5 model fine-tuned on Squad



# **1. Reproduce the probing tasks**

## **1.1 Use modified `jiant` library**
We modified some codes of the [original jiant library](https://github.com/nyu-mll/jiant) in order to achieve our desired functions that aren't supported by `jiant`. For more details please see our pdf report.

First, we will clone the modified jiant and install libraries we need for this code. We assume that this Notebook is run in a local clone of the /How-Does-Bert-Answer-QA-DLP2021/ repository

In [ ]:
%cd /src/probing-tasks/jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./

Restart runtime after installing libs

## **1.2 Set up the Edge Probing tasks**
After preprocessing and generating the Edge Probing data for all the tasks (see report for details), these data was uploaded to our github and will be used here. Next, we will create the corresponding task configs.

Because the tasks QUES and SUP are not supported by jiant, we added new task QUES to the jiant library (see report). The task SUP has the same jiant format structure as COREF, therefore we will reuse the default COREF task in jiant to probe SUP task.

Make sure to change the path to your local repository accordingly.

In [1]:
%cd /home/peter/Documents/Deep Learning/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/jiant/

import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

def init_task_config(task_name):
  jiant_task = task_name
  if(task_name == "sup-squad" or task_name == "sup-babi"):
    jiant_task = "coref"  # use coref task to probe supporting facts task because of the analog structure of jiant EP json format

  os.makedirs("../tasks/configs/", exist_ok=True)
  os.makedirs(f"../tasks/data/{task_name}", exist_ok=True)

  py_io.write_json({
    "task": jiant_task,
    "paths": {
      "train": f"../data/{task_name}/train.jsonl",
      "val":   f"../data/{task_name}/val.jsonl",
    },
    "name": jiant_task
  }, f"../tasks/configs/{task_name}_config.json")

/home/peter/Documents/Deep Learning/How-Does-Bert-Answer-QA-DLP2021/src/probing-tasks/jiant


In [3]:
task_names = [
              "ner", 
              #"semeval", 
              #"coref",     
              #"sup-squad", 
              #"ques"
              #"sup-babi",
              #"sup-hotpot",
             ]  

for task_name in task_names:
  init_task_config(task_name)

## **1.3 Download BERT models**
Next, we download a `bert-base-uncased` and a `bert-base-uncased-squad-v1` model



In [13]:
import jiant.proj.main.export_model as export_model

models = [
          "bert-base-uncased", 
          "csarron/bert-base-uncased-squad-v1"
          ]
          
for model in models:
  export_model.export_model(
      hf_pretrained_model_name_or_path=model,
      output_base_path=f"../tasks/models/{model}",
  )


Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForPreTraining: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at cs

## **1.4 Tokenize and cache**
With the model and data ready, we can now tokenize and cache the inputs features for our task. This converts the input examples to tokenized features ready to be consumed by the model, and saved them to disk in chunks.

In [11]:
import jiant.shared.caching as caching
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

# Tokenize and cache each task
def tokenize(task_names, models):
  for task_name in task_names:
    for model in models:
      tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
          task_config_path=f"../tasks/configs/{task_name}_config.json",
          hf_pretrained_model_name_or_path=model,
          output_dir=f"../tasks/cache/{task_name}",
          phases=["train", "val"],
          max_seq_length=384,
      ))

tokenize(task_names, models)

NerTask
  [train]: ../configs/../data/ner/train.jsonl
  [val]: ../configs/../data/ner/val.jsonl
Tokenizing: 100%|██████████| 1201/1201 [00:05<00:00, 206.87it/s]
NerTask
  [train]: ../configs/../data/ner/train.jsonl
  [val]: ../configs/../data/ner/val.jsonl
Tokenizing: 100%|██████████| 1201/1201 [00:05<00:00, 213.98it/s]


We can inspect the first examples of the first chunk of each task.

In [ ]:
row = caching.ChunkedFilesDataCache(f"../tasks/cache/{task_names[0]}/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)
print(row.spans)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
#print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

[  101 24111  1005  1055  2197  2420  1998  2054  2003  2056  2002  2001
  2066   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

## **1.4 Write a run config**
Here we are going to write what we call a `jiant_task_config`. This configuration file basically defines a lot of the subtleties of our training pipeline, such as what tasks we will train on, do evaluation on, batch size for each task. We use a helper `Configurator` to write out a `jiant_task_container_config`.

In [5]:
import jiant.proj.main.scripts.configurator as configurator

def create_jiant_task_config(task_name):
  jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
      task_config_base_path="../tasks/configs",
      task_cache_base_path="../tasks/cache",
      train_task_name_list=[task_name],
      val_task_name_list=[task_name],
      train_batch_size=8,
      eval_batch_size=2,
      epochs=5,
      num_gpus=1,
  ).create_config()
  os.makedirs("../tasks/run_configs/", exist_ok=True)
  py_io.write_json(jiant_run_config, f"../tasks/run_configs/{task_name}_run_config.json")
  #display.show_json(jiant_run_config)

## **1.5 Start training**
We will run probing each layers of each model for each task and the visualize the macro averaged F1 over all layers.

In [6]:
# You may restart runtime when you modify the jiant source code

import jiant.proj.main.runscript as main_runscript

def run_probing_task(task_name, model_name="bert-base-uncased", num_layers=1):
  run_args = main_runscript.RunConfiguration(
      jiant_task_container_config_path=f"../tasks/run_configs/{task_name}_run_config.json",
      output_dir=f"../tasks/runs/{task_name}",
      hf_pretrained_model_name_or_path=model_name,
      model_path=f"../tasks/models/{model_name}/model/model.p",
      model_config_path=f"../tasks/models/{model_name}/model/config.json",
      learning_rate=1e-3,
      eval_every_steps=1000,
      do_train=True,
      do_val=True,
      do_save=True,
      force_overwrite=True,
      num_hidden_layers=num_layers,
  )
  return main_runscript.run_loop(run_args)

In [7]:
import os
import matplotlib.pyplot as plt

num_layers = list(range(1, 13, 2))      # from 1 to 12 layers

def plot_task(model, task, linestyle):
  y = [float(results[model][task][n_layers]['f1_macro']) for n_layers in num_layers]
  plt.plot(num_layers, y,linestyle, label=model)
  plt.legend()
  plt.suptitle(task)

model_to_linestyle = {
    "bert-base-uncased": ":g", 
    "csarron/bert-base-uncased-squad-v1": "-y",

}

os.makedirs(f'../tasks/results', exist_ok=True)

In [ ]:
result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}

for model in models:
  results[model] = {}
  for task in task_names:
    results[model][task] = {}
    for n_layers in num_layers:
      results[model][task][n_layers] = {}
      create_jiant_task_config(task)
      probing_output = run_probing_task(task, model, n_layers)
      acc = str(probing_output[task]["metrics"]["minor"]["acc"])
      f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
      result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
      results[model][task][n_layers]['acc'] = acc
      results[model][task][n_layers]['f1_macro'] = f1_macro


In [ ]:
print(result)
plot_task('csarron/bert-base-uncased-squad-v1', 'sup-squad', '-y')
plt.show()

model	task	layer	acc	f1_macro
csarron/bert-base-uncased-squad-v1	coref	1	0.5476190476190477	0.47527472527472525
csarron/bert-base-uncased-squad-v1	coref	2	0.5476190476190477	0.3548387096774194
csarron/bert-base-uncased-squad-v1	coref	3	0.5714285714285714	0.33333333333333337
csarron/bert-base-uncased-squad-v1	coref	4	0.6904761904761905	0.33333333333333337
csarron/bert-base-uncased-squad-v1	coref	5	0.5	0.0
csarron/bert-base-uncased-squad-v1	coref	6	0.8571428571428571	0.4615384615384615
csarron/bert-base-uncased-squad-v1	coref	7	0.5	0.0
csarron/bert-base-uncased-squad-v1	coref	8	0.8571428571428571	0.4615384615384615
csarron/bert-base-uncased-squad-v1	coref	9	0.8571428571428571	0.4615384615384615
csarron/bert-base-uncased-squad-v1	coref	10	0.8571428571428571	0.4615384615384615
csarron/bert-base-uncased-squad-v1	coref	11	0.8571428571428571	0.4615384615384615
csarron/bert-base-uncased-squad-v1	coref	12	0.8571428571428571	0.4615384615384615



In [14]:
# Probe NER task with bert-base and bert-squad and plot macro f1 score
import json

result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}
task = "ner"

for model in models:
  results[model] = {}
  results[model][task] = {}
  for n_layers in num_layers:
    results[model][task][n_layers] = {}
    create_jiant_task_config(task)
    probing_output = run_probing_task(task, model, n_layers)
    acc = str(probing_output[task]["metrics"]["minor"]["acc"])
    f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
    result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
    results[model][task][n_layers]['acc'] = acc
    results[model][task][n_layers]['f1_macro'] = f1_macro

with open(f'../tasks/results/{task}.json', 'w') as f:
  json.dump(results, f)

for model in models:
  plot_task(model, task, model_to_linestyle[model])
plt.show()

l/model.p
  model_config_path: ../models/bert-base-uncased/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 3
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 0.001
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
Using seed: 3828254525
{
  "jiant_task_container_config_path": "../run_configs/ner_run_config.json",
  "output_dir": "../runs/ner",
  "hf_pretrained_model_name_or_path": "bert-base-uncased",
  "model_path": "../models/bert-base-uncased/model/model.p",
  "model_config_path"

RuntimeError: CUDA out of memory. Tried to allocate 54.00 MiB (GPU 0; 3.82 GiB total capacity; 2.22 GiB already allocated; 75.75 MiB free; 2.26 GiB reserved in total by PyTorch)

In [ ]:
# Probe SEMEVAL task with bert-base and bert-squad and plot macro f1 score
import json

result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}
task = "semeval"

for model in models:
  results[model] = {}
  results[model][task] = {}
  for n_layers in num_layers:
    results[model][task][n_layers] = {}
    create_jiant_task_config(task)
    probing_output = run_probing_task(task, model, n_layers)
    acc = str(probing_output[task]["metrics"]["minor"]["acc"])
    f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
    result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
    results[model][task][n_layers]['acc'] = acc
    results[model][task][n_layers]['f1_macro'] = f1_macro

with open(f'../tasks/results/{task}.json', 'w') as f:
  json.dump(results, f)

for model in models:
  plot_task(model, task, model_to_linestyle[model])
plt.show()

In [ ]:
# Probe COREF task with bert-base and bert-squad and plot macro f1 score
import json

result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}
task = "coref"

for model in models:
  results[model] = {}
  results[model][task] = {}
  for n_layers in num_layers:
    results[model][task][n_layers] = {}
    create_jiant_task_config(task)
    probing_output = run_probing_task(task, model, n_layers)
    acc = str(probing_output[task]["metrics"]["minor"]["acc"])
    f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
    result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
    results[model][task][n_layers]['acc'] = acc
    results[model][task][n_layers]['f1_macro'] = f1_macro

with open(f'../tasks/results/{task}.json', 'w') as f:
  json.dump(results, f)

for model in models:
  plot_task(model, task, model_to_linestyle[model])
plt.show()

In [ ]:
# Probe SUP-SQUAD task with bert-base and bert-squad and plot macro f1 score
import json

result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}
task = "sup-squad"

for model in models:
  results[model] = {}
  results[model][task] = {}
  for n_layers in num_layers:
    results[model][task][n_layers] = {}
    create_jiant_task_config(task)
    probing_output = run_probing_task(task, model, n_layers)
    acc = str(probing_output[task]["metrics"]["minor"]["acc"])
    f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
    result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
    results[model][task][n_layers]['acc'] = acc
    results[model][task][n_layers]['f1_macro'] = f1_macro

with open(f'../tasks/results/{task}.json', 'w') as f:
  json.dump(results, f)

for model in models:
  plot_task(model, task, model_to_linestyle[model])
plt.show()

In [ ]:
# Probe QUES task with bert-base and bert-squad and plot macro f1 score to compare the models
import json

result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}
task = "ques"

for model in models:
  results[model] = {}
  results[model][task] = {}
  for n_layers in num_layers:
    results[model][task][n_layers] = {}
    create_jiant_task_config(task)
    probing_output = run_probing_task(task, model, n_layers)
    acc = str(probing_output[task]["metrics"]["minor"]["acc"])
    f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
    result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
    results[model][task][n_layers]['acc'] = acc
    results[model][task][n_layers]['f1_macro'] = f1_macro

with open(f'../tasks/results/{task}.json', 'w') as f:
  json.dump(results, f)

for model in models:
  plot_task(model, task, model_to_linestyle[model])
plt.show()

In [ ]:
# Probe SUP-BABI task with bert-base and bert-squad and plot macro f1 score
import json

result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}
task = "sup-babi"

for model in models:
  results[model] = {}
  results[model][task] = {}
  for n_layers in num_layers:
    results[model][task][n_layers] = {}
    create_jiant_task_config(task)
    probing_output = run_probing_task(task, model, n_layers)
    acc = str(probing_output[task]["metrics"]["minor"]["acc"])
    f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
    result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
    results[model][task][n_layers]['acc'] = acc
    results[model][task][n_layers]['f1_macro'] = f1_macro

with open(f'../tasks/results/{task}.json', 'w') as f:
  json.dump(results, f)

for model in models:
  plot_task(model, task, model_to_linestyle[model])
plt.show()

In [ ]:
# Probe SUP-HOTPOT task with bert-base and bert-squad and plot macro f1 score to compare the models
import json

result = "model\ttask\tlayer\tacc\tf1_macro\n"
results = {}
task = "sup-hotpot"

for model in models:
  results[model] = {}
  results[model][task] = {}
  for n_layers in num_layers:
    results[model][task][n_layers] = {}
    create_jiant_task_config(task)
    probing_output = run_probing_task(task, model, n_layers)
    acc = str(probing_output[task]["metrics"]["minor"]["acc"])
    f1_macro = str(probing_output[task]["metrics"]["minor"]["f1_macro"])
    result += model + "\t" + task + "\t" + str(n_layers) + "\t" + acc + "\t" + f1_macro + "\n"
    results[model][task][n_layers]['acc'] = acc
    results[model][task][n_layers]['f1_macro'] = f1_macro

with open(f'../tasks/results/{task}.json', 'w') as f:
  json.dump(results, f)

for model in models:
  plot_task(model, task, model_to_linestyle[model])
plt.show()

# **2. Experiment with BERT (todo: which type?) trained on (todo: which dataset?) dataset**

# **3. Experiment with T5 model fine-tuned on Squad**